# Create A Playlist Based On Featured Playlists
*A Project by Shraddha Pattnaik*

### Steps done to create a playlist:
1. Import necessary libraries
2. Provide your client id, client secret and redirect uri
3. Authorization and get token
4. Data Extraction:
    a. User's top tracks
    b. Featured Playlists
5. Fetch audio features of the tracks
6. Analyze the features using Random Forest Regressor
7. Make a playlist using top 2-3 featured playlists closest to user's top tracks 
8. Create a new playlist and add the songs

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

import random
from functools import reduce

import warnings
warnings.filterwarnings('ignore')

In [2]:
client_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'
client_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
redirect_uri='http://localhost:5000'

username = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [3]:
#FINDING USER'S TOP TRACKS
scope = 'user-top-read'

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope=scope,
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        username=username
    )
)

In [4]:
results = sp.current_user_top_tracks(limit=10, offset=0,time_range='medium_term')

In [5]:
#converting it into data frame
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_topfavourite = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_topfavourite

,track_name,album,track_id,artist,duration,popularity
0,Haaray,Haaray,2LCvypYL9a21Hln4A4EdWU,Abdul Hannan,184000,62
1,Tu Aake Dekhle,The Carnival,0yCWDaAgOtg6TKlNCg9rwA,King,270000,82
2,Junoon,Junoon,4roFMBSQp7W4fYVxKMmhA5,Mitraz,191111,31
3,kya karoon?,Genesis 1:1,6nXwthh92R2HlnyBWtHgHP,Zaeden,200275,60
4,Die For You,Starboy,2LBqCSwhJGcFQeTHMVGwy3,The Weeknd,260253,91
5,sure thing (sped up),sure thing (sped up),6fBiLOkPcQ3iFUpmIv88fQ,accelerate,113053,73
6,,,7Els4IIFYa4P4RxBkZkxdd,,0,0
7,The Great War,Midnights (3am Edition),3UMrglJeju5yWyYIW6o99b,Taylor Swift,240355,81
8,Lost in the Fire (feat. The Weeknd),Hyperion,2vXKRlJBXyOcvZYTdNeckS,Gesaffelstein,202093,85
9,Akhiyaan,Akhiyaan,2l3dFxtoSMgMQVyRIUHjpz,Mitraz,191868,75


In [6]:
def fetch_audio_features(sp, df):
    playlist = df[['track_id','track_name']] 
    index = 0
    audio_features = []
    
    # Make the API request
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness'],
                              features['mode']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness','duration_ms', 'key',
                                                             'valence', 'speechiness', 'mode'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [7]:
df_fav = fetch_audio_features(sp, df_topfavourite)
df_fav

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
track_name,,,,,,,,,,,,,
Haaray,2LCvypYL9a21Hln4A4EdWU,0.716,0.6340,0.290,90.019,0.000073,-12.098,0.0987,184000,9,0.2810,0.0332,1
Tu Aake Dekhle,0yCWDaAgOtg6TKlNCg9rwA,0.770,0.5150,0.840,96.012,0.004520,-4.022,0.1450,270000,11,0.1920,0.1070,1
Junoon,4roFMBSQp7W4fYVxKMmhA5,0.607,0.2000,0.593,107.988,0.000000,-7.982,0.0833,191111,4,0.2110,0.0378,0
kya karoon?,6nXwthh92R2HlnyBWtHgHP,0.723,0.6450,0.560,145.022,0.000007,-8.324,0.2280,200276,5,0.6190,0.0350,0
Die For You,2LBqCSwhJGcFQeTHMVGwy3,0.586,0.1110,0.525,133.629,0.000000,-7.163,0.1340,260253,1,0.5080,0.0615,0
sure thing (sped up),6fBiLOkPcQ3iFUpmIv88fQ,0.661,0.0435,0.511,192.632,0.020100,-8.675,0.1170,113054,9,0.5520,0.2430,0
,7Els4IIFYa4P4RxBkZkxdd,0.586,0.8030,0.562,93.982,0.000006,-7.399,0.0630,261702,9,0.4360,0.0267,1
The Great War,3UMrglJeju5yWyYIW6o99b,0.573,0.2190,0.741,96.016,0.000022,-8.987,0.0842,240356,5,0.5540,0.0353,1
Lost in the Fire (feat. The Weeknd),2vXKRlJBXyOcvZYTdNeckS,0.658,0.0933,0.671,100.966,0.000927,-12.210,0.1150,202093,2,0.1660,0.0363,1


In [8]:
#FETCHING FEATURED PLAYLISTS
response = sp.featured_playlists()
# check the format of the data 
response

{'message': "Editor's picks",
 'playlists': {'href': 'https://api.spotify.com/v1/browse/featured-playlists?country=IN&timestamp=2023-05-24T16%3A19%3A20&offset=0&limit=20',
  'items': [{'collaborative': False,
    'description': 'Where Bollywood meets the hood. Cover: Badshah',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWU2tlzlUMZTA'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWU2tlzlUMZTA',
    'id': '37i9dQZF1DWU2tlzlUMZTA',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f00000003d624ceed3a2cc0c89df10ecf',
      'width': None}],
    'name': 'Bollyhood',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTY4MTc5NjI2MiwwMDAwMDAwMDc5MzJiZTR

In [9]:
def featured_playlists(sp):
    pl_id = []
    name = []
    num_tracks = []
 # For looping through the API request  
    response = sp.featured_playlists()
    playlists = response['playlists']
    for i, items in enumerate(playlists['items']):
        pl_id.append(items['id'])
        name.append(items['name'])
        num_tracks.append(items['tracks']['total'])

# Create the final df   
    df_playlists = pd.DataFrame({"id":pl_id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [10]:
df_playlists = featured_playlists(sp)
df_playlists

,id,name,#tracks
0,37i9dQZF1DWU2tlzlUMZTA,Bollyhood,23
1,37i9dQZF1DX4Cmr6Ex5w24,Rahman Ruling 90s,75
2,37i9dQZF1DWVDCraF986xg,Punjabi X,50
3,37i9dQZF1DWZdcdjsv83gQ,Telugu Party Time,75
4,37i9dQZF1DX1dCsSMSXSsP,Pop Shots,40
5,37i9dQZF1DX0nA91dV2ts4,Tamil Party Time,81
6,37i9dQZF1DWVDvBpGQbzXj,Music Shuttle,46
7,37i9dQZF1DWTqYqGLu7kTX,Hip Te Hop,80
8,37i9dQZF1DWY7EoqMbT7wZ,90s Romance Tamil,75
9,37i9dQZF1DX5EEpa9ekxRI,All Out 90s Telugu,75


In [11]:
def fetch_playlist_tracks(sp, playlistsid): 
    offset = 0
    tracks = []
    # Make the API request
    while True:
            content = sp.playlist_tracks( playlistsid, fields=None, limit=100, offset=offset, market=None)
            tracks += content['items']
        
            if content['next'] is not None:
                offset += 100
            else:
                break
    
    track_id = []
    track_name = []
    
    for track in tracks:
        track_id.append(track['track']['id'])
        track_name.append(track['track']['name'])
    
# Create the final df
    df_playlists_tracks = pd.DataFrame({"track_id":track_id, "track_name": track_name})
    return df_playlists_tracks

In [12]:
# Test the function
fetch_playlist_tracks(sp,'37i9dQZF1DXb5Hc9BmAT8t')

,track_id,track_name
0,5Tn2aBp45QypSmCpmhKtJN,Kalalo Kooda
1,4Te9tTtG4ipT6x6G69DTJd,"Kumkumala (From ""Brahmastra (Telugu)"")"
2,3Jjtn4IEN0cjPcfpvIDsfn,Ye Kannulu Chudani Chitrame
3,1kYg9IZJ9QwrnHaJztmK5n,Emo Emo
4,18JVTOKs9ghqPLtkMeqffL,"Baguntundhi Nuvvu Navvithe (From ""Atithi Devob..."
...,...,...
70,25aFmFxEx6jCvovZig4XIz,Egiregire
71,3DguE3UzTV02VkzKhOjz8g,Yegire Yegire
72,5LGx7PoLBkxJtZiocex02H,Ninne Ninne
73,76BpZZ6zc4tOWbM1tdl3yM,Chinni Chinni Chinukulu


In [13]:
def fetch_audio_features(sp, playlist_id):
    playlist = fetch_playlist_tracks(sp, playlist_id)
    audio_features = []
    
    for i in playlist['track_id']:
        audio_features += sp.audio_features(i)
   
    
    # Create an empty list to feed in different charactieritcs of the tracks
    features_list = []
    #Create keys-values of empty lists inside nested dictionary for album
    for features in audio_features:
        if features == None:
            features_list.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
        else:
            features_list.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    
    df_audio_features = pd.DataFrame(features_list, columns=['danceability', 'acousticness', 'energy','tempo', 
                                                             'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key',
                                                             'valence', 'speechiness'])
    
    # Create the final df, using the 'track_id' as index for future reference
    df_playlist_audio_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_audio_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_audio_features

In [14]:
fetch_audio_features(sp, '37i9dQZF1DXb5Hc9BmAT8t')

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Kalalo Kooda,5Tn2aBp45QypSmCpmhKtJN,0.724,0.6510,0.596,108.011,0.000000,-6.580,0.0692,246806,5,0.605,0.0289
"Kumkumala (From ""Brahmastra (Telugu)"")",4Te9tTtG4ipT6x6G69DTJd,0.000,0.0000,0.000,0.000,0.000000,0.000,0.0000,0,0,0.000,0.0000
Ye Kannulu Chudani Chitrame,3Jjtn4IEN0cjPcfpvIDsfn,0.457,0.6770,0.473,109.912,0.000000,-8.639,0.1530,293934,4,0.401,0.0486
Emo Emo,1kYg9IZJ9QwrnHaJztmK5n,0.266,0.8580,0.539,199.845,0.000210,-7.841,0.3380,242413,0,0.557,0.0402
"Baguntundhi Nuvvu Navvithe (From ""Atithi Devobhava"")",18JVTOKs9ghqPLtkMeqffL,0.760,0.5040,0.443,123.959,0.000797,-9.128,0.0957,222581,6,0.632,0.0426
...,...,...,...,...,...,...,...,...,...,...,...,...
Egiregire,25aFmFxEx6jCvovZig4XIz,0.646,0.5210,0.741,88.468,0.000000,-5.215,0.1250,190802,2,0.537,0.0498
Yegire Yegire,3DguE3UzTV02VkzKhOjz8g,0.391,0.7220,0.476,95.005,0.000009,-8.980,0.1060,225405,10,0.733,0.0375
Ninne Ninne,5LGx7PoLBkxJtZiocex02H,0.810,0.0536,0.369,105.028,0.011100,-10.899,0.1500,331733,5,0.693,0.0371


In [15]:
def fetch_audio_features_mean(sp, playlist_id):
    Playlist = fetch_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [16]:
# Test the function
df = fetch_audio_features_mean(sp, '37i9dQZF1DX3wwp27Epwn5')
df.head(10)

,37i9dQZF1DX3wwp27Epwn5
danceability,0.712140
acousticness,0.147235
energy,0.833660
tempo,119.655220
instrumentalness,0.002051
loudness,-4.910820
liveness,0.194226
duration_ms,224930.880000
key,5.240000
valence,0.632560


In [17]:
# Merge them together
merged_df = []
# Loop through the filenames to populate dataframes with different dataframes 
for i in df_playlists['id']:
    merged_df.append(fetch_audio_features_mean(sp, i))

In [18]:
# ANALYZE THE FEATURE IMPORTANCE
from sklearn.ensemble import RandomForestRegressor

X = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), merged_df)
Y = pd.DataFrame(df_fav.median(), columns= ['fav_playlist'])
Y = Y.drop('mode')

forest = RandomForestRegressor(random_state=42, max_depth=5, max_features=10) 
forest.fit(X,Y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature rankings
print("Playlist ranking:")
  
for f in range(len(importances)):
    print("%d. %s %f " % (f + 1, 
            X.columns[f], 
            importances[indices[f]]))

Playlist ranking:
1. 37i9dQZF1DWU2tlzlUMZTA 0.110473 
2. 37i9dQZF1DX4Cmr6Ex5w24 0.110055 
3. 37i9dQZF1DWVDCraF986xg 0.110005 
4. 37i9dQZF1DWZdcdjsv83gQ 0.099663 
5. 37i9dQZF1DX1dCsSMSXSsP 0.098009 
6. 37i9dQZF1DX0nA91dV2ts4 0.080532 
7. 37i9dQZF1DWVDvBpGQbzXj 0.078538 
8. 37i9dQZF1DWTqYqGLu7kTX 0.070876 
9. 37i9dQZF1DWY7EoqMbT7wZ 0.070202 
10. 37i9dQZF1DX5EEpa9ekxRI 0.069549 
11. 37i9dQZF1DX5HjYnbxXZwh 0.062013 
12. 37i9dQZF1DWTAtTdFMiJYK 0.040086 


In [19]:
# MAKING A PLAYLIST USING TOP 3
frames = [fetch_audio_features(sp, '37i9dQZF1DWU2tlzlUMZTA'),
          fetch_audio_features(sp, '37i9dQZF1DX4Cmr6Ex5w24'),
          fetch_audio_features(sp, '37i9dQZF1DWVDCraF986xg')
         ]
df = pd.concat(frames)
df

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
"Sher (From ""Selfiee"")",7tO6bKdh9fzsTOP139HV3E,0.551,0.000801,0.812,87.450,0.000127,-5.978,0.0472,156007,0,0.550,0.0305
Paisa Hai Toh,5kwldaZvtbiTmRYsmlyxAN,0.672,0.205000,0.759,150.042,0.000000,-5.220,0.2910,188587,4,0.794,0.0352
"Asli Action Chaalu (Theme Song) [From ""An Action Hero""]",7wTaexlHXGPcx3ORPLzRvv,0.761,0.151000,0.741,139.028,0.011700,-10.641,0.2040,184001,6,0.572,0.0936
Abhi Toh Party Shuru Hui Hai,0rUN8ascPVT9iiLzC5ocPw,0.806,0.008100,0.815,129.999,0.000023,-5.373,0.5860,179912,3,0.678,0.0599
Nyaay Yuddh Dharm Yuddh,2OlLzyE9YtysMoOjA8KkHK,0.762,0.093400,0.682,105.990,0.000000,-7.050,0.2760,142500,2,0.344,0.1350
...,...,...,...,...,...,...,...,...,...,...,...,...
"Ban Ja Rani (From ""Tumhari Sulu"")",7cQtGVoPCK9DlspeYjdHOA,0.813,0.206000,0.692,102.015,0.000000,-4.718,0.1240,225938,9,0.487,0.0577
Chhote Chhote Peg,3AQrBSjowqCFu2jiFFuGsv,0.831,0.328000,0.908,104.962,0.003710,-3.317,0.0770,203882,7,0.734,0.1010
Angreji Beat,0IhR7oLOe2ELVfbwmpLdBn,0.859,0.211000,0.998,119.967,0.001510,-1.501,0.0452,212627,9,0.274,0.3130


In [20]:
df = df.set_index(['track_id'])
df

,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_id,,,,,,,,,,,
7tO6bKdh9fzsTOP139HV3E,0.551,0.000801,0.812,87.450,0.000127,-5.978,0.0472,156007,0,0.550,0.0305
5kwldaZvtbiTmRYsmlyxAN,0.672,0.205000,0.759,150.042,0.000000,-5.220,0.2910,188587,4,0.794,0.0352
7wTaexlHXGPcx3ORPLzRvv,0.761,0.151000,0.741,139.028,0.011700,-10.641,0.2040,184001,6,0.572,0.0936
0rUN8ascPVT9iiLzC5ocPw,0.806,0.008100,0.815,129.999,0.000023,-5.373,0.5860,179912,3,0.678,0.0599
2OlLzyE9YtysMoOjA8KkHK,0.762,0.093400,0.682,105.990,0.000000,-7.050,0.2760,142500,2,0.344,0.1350
...,...,...,...,...,...,...,...,...,...,...,...
7cQtGVoPCK9DlspeYjdHOA,0.813,0.206000,0.692,102.015,0.000000,-4.718,0.1240,225938,9,0.487,0.0577
3AQrBSjowqCFu2jiFFuGsv,0.831,0.328000,0.908,104.962,0.003710,-3.317,0.0770,203882,7,0.734,0.1010
0IhR7oLOe2ELVfbwmpLdBn,0.859,0.211000,0.998,119.967,0.001510,-1.501,0.0452,212627,9,0.274,0.3130


In [21]:
Y = Y.squeeze('columns')
Y = Y.sort_index( axis=0, level=None, ascending=True, inplace=False, kind='quicksort')

In [22]:
df1= df.subtract(Y, axis='columns') 

df1= df1.divide(Y, axis='columns') 

# Add all the score 
df1['variation'] = df1.sum(axis=1)

# take the absoulte of the variation
df1['variation'] = df1['variation'].abs()

df1

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
7tO6bKdh9fzsTOP139HV3E,-0.996177,-0.164519,-0.224558,0.406061,1.665268,-1.0,-0.558259,-0.296665,-0.176788,-0.129794,0.534170,0.941261
5kwldaZvtbiTmRYsmlyxAN,-0.021480,0.018954,-0.062617,0.314286,-1.000000,-0.2,1.723444,-0.385846,-0.049933,0.493052,1.214784,2.044644
7wTaexlHXGPcx3ORPLzRvv,-0.279236,0.153904,-0.085412,0.283117,244.540399,0.2,0.909219,0.251956,1.526316,0.383453,0.595537,248.479252
0rUN8ascPVT9iiLzC5ocPw,-0.961337,0.222138,-0.105736,0.411255,-0.506821,-0.4,4.484324,-0.367845,0.616734,0.293606,0.891213,4.577532
2OlLzyE9YtysMoOjA8KkHK,-0.554177,0.155421,-0.291695,0.180952,-1.000000,-0.6,1.583060,-0.170539,2.643725,0.054695,-0.040446,1.960996
...,...,...,...,...,...,...,...,...,...,...,...,...
7cQtGVoPCK9DlspeYjdHOA,-0.016706,0.232752,0.123039,0.198268,-1.000000,0.8,0.160505,-0.444909,0.557355,0.015140,0.358438,0.983883
3AQrBSjowqCFu2jiFFuGsv,0.565632,0.260045,0.013408,0.572294,76.859391,0.4,-0.279364,-0.609742,1.726046,0.044466,1.047420,80.599598
0IhR7oLOe2ELVfbwmpLdBn,0.007160,0.302502,0.056876,0.728139,30.689402,0.8,-0.576977,-0.823401,7.448043,0.193779,-0.235704,38.589817


In [26]:
# Now we have the variation, we will take the songs with the least variation 
df2 = df1.nsmallest(10,'variation')
df2

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,variation
track_id,,,,,,,,,,,,
6erhWA07TRTqtISu5ODMlL,-0.365155,0.102350,0.662901,-0.328139,-1.000000,0.2,-0.346748,0.391611,0.179487,-0.054486,0.623431,0.065254
4g9VvOS1WpFp2FZVSzvIZA,-0.451074,0.144807,1.054955,-0.104762,-0.961595,0.4,0.020122,0.124419,-0.236167,0.035042,-0.185495,0.159749
05z1MwVygW1kGEatTSoLl9,1.033413,-0.176649,0.427148,-0.348918,-1.000000,-1.0,-0.555452,0.624213,0.398111,0.670944,0.126918,0.199728
09vHbIezp9NP31CZWysk3v,-0.609547,0.191812,-0.049435,0.258874,-1.000000,0.8,-0.228825,-0.408436,0.376518,-0.064915,0.944212,0.210259
6078vTOfNPOHZDuyzzsMLx,0.622912,-0.325246,0.672574,-0.387013,-1.000000,-0.2,0.431914,0.402082,-0.036437,-0.003717,0.046025,0.223094
2zQeAeH3q0K7U2AWEcVOsF,-0.937947,0.413192,-0.165209,0.561905,-0.850367,-0.8,-0.402901,-0.770928,1.726046,0.094668,0.821478,0.310064
27iITPapIoJUhBSy3EhjL8,-0.436754,0.188779,1.041917,0.603463,-0.967261,-0.6,-0.774450,-0.233131,0.209177,-0.005130,1.306834,0.333443
4xZqoLPxeIUItAiF16H6Ve,1.343675,-0.246399,0.267528,-0.636364,-1.000000,-0.6,0.459991,0.722925,-0.174089,-0.055252,-0.461646,0.379631
5I2wbDvSI98HLVB42YO9sU,0.112172,0.191812,0.070500,0.380087,-1.000000,0.2,-0.194197,-0.521972,0.117409,-0.054526,0.263598,0.435118


In [27]:
#Create playlist
scope = 'playlist-modify-public'

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope=scope,
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        username=username
    )
)

def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [28]:
create_playlist(sp, username, 'Test', 'Test playlist created using python!')

In [43]:
playlists = sp.user_playlists(username)

pl = list(playlists['items'])[0]

print(pl['id'], '\t', pl['name'], '\t', pl['tracks']['total'])

5MYf4yLS182DCmtsYncap8 	 Test 	 0


In [48]:
def add_to_playlist(sp, username, playlist_id, playlist_tracks):
    index = 0
    results = []
    
    while index < len(playlist_tracks):
        results += sp.user_playlist_add_tracks(username, playlist_id, tracks = playlist_tracks[index:index + 50])
        index += 50

In [49]:
list_track = df2.index

In [50]:
add_to_playlist(sp, username, '5MYf4yLS182DCmtsYncap8', list_track)

In [51]:
playlists = sp.user_playlists(username)

pl = list(playlists['items'])[0]

print(pl['id'], '\t', pl['name'], '\t', pl['tracks']['total'])

5MYf4yLS182DCmtsYncap8 	 Test 	 10
